<a href="https://colab.research.google.com/github/dawood5253/Movie-Recommendation-System/blob/main/Movie_Recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Popularity based Recommender

In [ ]:
#1.Create a popularity-based recommender system at a genre level. The user will input a genre (g), minimum rating threshold (t) for a movie,and no. of recommendations(N) for which it should be recommended top N movies which are most popular within that genre (g) ordered by ratings in descending order where each movie has at least (t) reviews

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [ ]:
mov=pd.read_csv('movies.csv')
mov.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rat=pd.read_csv('ratings.csv')
rat.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
df = pd.merge(rat, mov, on='movieId')
df.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama
5,31,16,4.0,832397760,Casino (1995),Crime|Drama
6,47,16,4.0,853361808,Casino (1995),Crime|Drama
7,60,16,4.0,1162941904,Casino (1995),Crime|Drama
8,62,16,4.5,1267576909,Casino (1995),Crime|Drama
9,88,16,3.5,1180879072,Casino (1995),Crime|Drama


In [ ]:
df['genres'] = df['genres'].str.split('|')
df = df.explode('genres')

In [ ]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
0,1,16,4.0,1217897793,Casino (1995),Drama
1,9,16,4.0,842686699,Casino (1995),Crime
1,9,16,4.0,842686699,Casino (1995),Drama
2,12,16,1.5,1144396284,Casino (1995),Crime


In [ ]:
g=df.groupby('genres')

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

#Creating the function for calculating the top movies
def poprec(gen, min, nrec):
    filt = g.get_group(gen).reset_index().drop('index', axis=1)
    ratings = pd.DataFrame(filt.groupby('title')['rating'].mean())
    ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
    sys = ratings[ratings['num of ratings'] > min].sort_values('num of ratings', ascending=False).head(nrec)
    return sys



# Creating widgets
gen_dropdown = widgets.Dropdown(
    options=g.groups.keys(),
    description='Genre:'
)

min_text = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Minimum Ratings:'
)

nrec_text = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Number of Recommendations:'
)

# Output widget for displaying recommendations
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        genre = gen_dropdown.value
        min_ratings = min_text.value
        n_rec = nrec_text.value
        recommendations = poprec(genre, min_ratings, n_rec)
        print(recommendations)

button = widgets.Button(description="Get Recommendations")
button.on_click(on_button_clicked)

# Displaying widgets
display(gen_dropdown)
display(min_text)
display(nrec_text)
display(button)
display(output)


Dropdown(description='Genre:', options=('(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', …

IntSlider(value=0, description='Minimum Ratings:')

IntSlider(value=10, description='Number of Recommendations:', min=1)

Button(description='Get Recommendations', style=ButtonStyle())

Output()

#  Content-based recommender system

In [ ]:
movies=set(df['title'])

In [ ]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [ ]:
moviemat = df.pivot_table(index = 'userId', columns='title', values = 'rating')
moviemat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

def conrec(mov, nrec):
    if mov in movies:
        sel = moviemat[mov]
        siml = moviemat.corrwith(sel)
        corr = pd.DataFrame(siml, columns=['Correlation'])
        corr.dropna(inplace=True)
        recs = corr.sort_values('Correlation', ascending=False).head(nrec)

        return recs
    else:
        print('Movie not Found!!')

# Assuming movies and moviemat are already defined

# Creating widgets
mov_text = widgets.Text(
    value='',
    placeholder='Enter movie name',
    description='Movie:'
)

nrec_text = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Number of Recommendations:'
)

# Output widget for displaying recommendations
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        movie = mov_text.value
        n_rec = nrec_text.value
        recommendations = conrec(movie, n_rec)
        if recommendations is not None:
            print(recommendations)
        else:
            print("Movie not found.")

button = widgets.Button(description="Get Recommendations")
button.on_click(on_button_clicked)

# Displaying widgets
display(mov_text)
display(nrec_text)
display(button)
display(output)






Text(value='', description='Movie:', placeholder='Enter movie name')

IntSlider(value=10, description='Number of Recommendations:', min=1)

Button(description='Get Recommendations', style=ButtonStyle())

Output()

# Collaborative Based Recommender

In [ ]:
df1 = pd.merge(rat, mov, on='movieId')

In [ ]:
df1.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [ ]:
moviematr = df1.pivot_table(index = 'userId', columns='title', values = 'rating')
moviematr

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df1 = df1.drop_duplicates(subset=['userId', 'title'])


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

def generate_recommendations(user, num_recommendations=5, k_similar_users=100):
    # Calculate Pearson correlation coefficients between users
    user_correlations = df1.pivot_table(index='userId', columns='title', values='rating').corrwith(df1.pivot_table(index='userId', columns='title', values='rating').loc[user], axis=1, method='pearson').dropna()

    # Sort similar users by correlation coefficient and select top k
    similar_users = user_correlations.nlargest(k_similar_users)

    # Find movies not rated by the target user
    user_movies = df1[df1['userId'] == user]['title']
    unrated_movies = df1[~df1['title'].isin(user_movies)]['title'].unique()

    # Generate recommendations based on similar users' ratings
    recommendations = {}
    for movie in unrated_movies:
        similar_ratings = df1[df1['title'] == movie].set_index('userId').reindex(similar_users.index)[['rating']]
        weighted_ratings = similar_ratings.mul(similar_users, axis=0)
        total_similarity = weighted_ratings.sum(axis=0)
        if not total_similarity.empty and total_similarity.iloc[0] != 0:
            recommendations[movie] = (weighted_ratings.sum(axis=0) / total_similarity).values[0]

    # Sort recommendations by predicted rating
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]

    df_recommendations = pd.DataFrame(sorted_recommendations, columns=['title', 'predicted_rating'])
    df_recommendations.drop('predicted_rating',axis=1,inplace=True)

    return df_recommendations

# Assuming df1 is already defined

# Creating widgets
user_dropdown = widgets.Dropdown(
    options=df1['userId'].unique(),
    description='User:'
)

num_rec_text = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:'
)

k_similar_users_text = widgets.IntSlider(
    value=100,
    min=1,
    max=500,
    step=1,
    description='Number of Similar Users:'
)

# Output widget for displaying recommendations
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        user = user_dropdown.value
        num_recommendations = num_rec_text.value
        k_similar_users = k_similar_users_text.value
        recommendations = generate_recommendations(user, num_recommendations, k_similar_users)
        if not recommendations.empty:
            display(recommendations)
        else:
            print("No recommendations found for this user.")

button = widgets.Button(description="Get Recommendations")
button.on_click(on_button_clicked)

# Displaying widgets
display(user_dropdown)
display(num_rec_text)
display(k_similar_users_text)
display(button)
display(output)


Dropdown(description='User:', options=(1, 9, 12, 24, 29, 31, 47, 60, 62, 88, 104, 109, 113, 122, 128, 137, 149…

IntSlider(value=5, description='Number of Recommendations:', max=20, min=1)

IntSlider(value=100, description='Number of Similar Users:', max=500, min=1)

Button(description='Get Recommendations', style=ButtonStyle())

Output()